In [1]:
from typing import Literal

from dotenv import load_dotenv  # type: ignore
from notebooks._utils import (
    celsius_to_fahrenheit,
    city_to_number,
    city_to_weather_condition,
)
from pydantic import Field  # type: ignore
from baml_agents import with_model
from baml_client.sync_client import b
import jupyter_black  # type: ignore
import stackprinter  # type: ignore

from baml_client import types
from baml_client import types as T
from baml_client.type_builder import TypeBuilder

stackprinter.set_excepthook()
load_dotenv()
jupyter_black.load()

b = with_model(b, "gpt-4.1-nano")

## Creating tools

Personally, I prefer the term "Actions," so I'll use "Tools" and "Actions" interchangeably.

Let's create a tool for an AI to use (Action). We'll be using Pydantic-model-as-a-function pattern rather than regular typed Python functions because that makes it easer to add parameter validation, type hints, and baml-specific (LLM-specific) information to the function parameters.

So instead of writing...:

In [2]:
from baml_agents import Result


def get_weather_info(
    city: str,
    measurement: Literal["celsius", "fahrenheit"] = "celsius",
) -> Result:
    """Get weather information for a given city."""
    return Result(
        content="The weather in London is 18 degrees celsius with windy conditions.",
        error=False,
    )

...we will do:

In [3]:
from baml_agents import Action


class GetWeatherInfo(Action):
    """Get weather information for a given city."""

    city: str
    measurement: Literal["celsius", "fahrenheit"] | None = None

    def run(self) -> Result:
        self.measurement = self.measurement or "celsius"
        c = city_to_number(self.city, -10, 35)
        condition = city_to_weather_condition(self.city)
        if self.measurement.lower() == "fahrenheit":
            c, u = celsius_to_fahrenheit(c), "fahrenheit"
        else:
            u = "celsius"
        content = f"The weather in {self.city} is {round(c, 1)} degrees {u} with {condition.lower()} conditions."
        return Result(content=content, error=False)

Let's try some cities:

In [4]:
print(GetWeatherInfo(city="New York", measurement="fahrenheit").run())
print(GetWeatherInfo(city="London").run())
print(GetWeatherInfo(city="Paris").run())

content='The weather in New York is 57.2 degrees fahrenheit with foggy conditions.' error=False
content='The weather in London is -7 degrees celsius with sunny conditions.' error=False
content='The weather in Paris is -10 degrees celsius with rainy conditions.' error=False


## Manually configuring baml

According to [12 Factor Agents](https://github.com/humanlayer/12-factor-agents), [tools are simply structured outputs](https://github.com/search?q=repo%3Ahumanlayer%2F12-factor-agents+Tools+are+just+structured+outputs&type=code).

That's why we'll just create a baml prompt template:

In [5]:
# This command shows file contents
!awk '/^class BamlCustomTools_GetWeatherInfo_Parameters/,/^}/' ../baml_src/notebooks/02_baml_custom_tools.baml
!awk '/^function BamlCustomTools_GetWeatherInfo/,/^}/' ../baml_src/notebooks/02_baml_custom_tools.baml

class BamlCustomTools_GetWeatherInfo_Parameters {
    city string
    measurement "celsius" | "fahrenheit" @description(#"default: celsius"#)
}
function BamlCustomTools_GetWeatherInfo(text: string) -> BamlCustomTools_GetWeatherInfo_Parameters {
  client Default
  prompt #"
    {{ text }}
    {{ ctx.output_format }}
  "#
}


Let's see the actual prompt that we will send to the LLM:

In [6]:
from baml_agents import display_prompt

question = "What's the weather like in the capital of Japan?"

request = b.request.BamlCustomTools_GetWeatherInfo(question)
display_prompt(request)

[system]
What's the weather like in the capital of Japan?
Answer in JSON using this schema:
{
  city: string,
  // default: celsius
  measurement: "celsius" or "fahrenheit",
}


Let's send it and run the result:

In [7]:
question = "What's the weather like in the capital of Japan?"

# Calling LLM
completion = b.BamlCustomTools_GetWeatherInfo(question)
print(completion)

city='Tokyo' measurement='celsius'


In [8]:
# Running the action
GetWeatherInfo.validate(completion).run()

Result(content='The weather in Tokyo is 26 degrees celsius with rainy conditions.', error=False)

## Automatically configuring baml

We'll be using the [dynamic features of baml](https://docs.boundaryml.com/guide/baml-advanced/dynamic-runtime-types).

In [9]:
# This command shows file contents
!awk '/^class BamlCustomTools_NextAction/,/^}/' ../baml_src/notebooks/02_baml_custom_tools.baml
!awk '/^function BamlCustomTools_GetNextAction/,/^}/' ../baml_src/notebooks/02_baml_custom_tools.baml

class BamlCustomTools_NextAction {
    @@dynamic
}
function BamlCustomTools_GetNextAction(goal: string) -> BamlCustomTools_NextAction {
    client Default
    prompt #"
        Please select the next best action to achieve the goal.
        
        <goal>
        {{ goal }}
        </goal>
        
        {{ ctx.output_format(or_splitter=" OR ", prefix="Answer in JSON format with exactly one next best action:\n") }}
    "#
}


To add fields from the Tool to a class, we must first know **which** class we want to add the fields to. This can be done in two ways:
#### Choosing Target Class - Method 1: Simple

It works by taking a look at the baml function annotation return type.

In [10]:
from baml_agents import ActionRunner

r = ActionRunner(TypeBuilder, b=b)
r.add_action(GetWeatherInfo)

question = "What's the weather like in the capital of Japan?"
request = r.b.request.BamlCustomTools_GetNextAction(question)

display_prompt(request)

[system]
Please select the next best action to achieve the goal.

‹goal›
What's the weather like in the capital of Japan?
‹/goal›

Answer in JSON format with exactly one next best action:
{
  chosen_action: {
    // Get weather information for a given city.
    action_id: "get_weather_info",
    city: string,
    measurement: "celsius" OR "fahrenheit" OR null,
  },
}


#### Choosing Target Class - Method 2

You will only need it if you're adding tools not to the root class, otherwise, skip this

... [code moved to the bottom of the file]

#### Choosing Target Class - Method 3

You will only need this so you could fall back to a simpler method if you encouter bugs or need more customization

... [code moved to the bottom of the file]

#### Let's send the request, get the completion and run the action:

In [11]:
question = "What's the weather like in the capital of Japan?"

r = ActionRunner(TypeBuilder)
r.add_action(GetWeatherInfo)

# Calling the LLM
action = b.BamlCustomTools_GetNextAction(
    question, baml_options={"tb": r.tb(T.BamlCustomTools_NextAction)}
)
print(action)

chosen_action={'action_id': 'get_weather_info', 'city': 'Tokyo', 'measurement': 'celsius'}


In [12]:
# Running the action chosen by LLM
r.run(action)

Result(content='The weather in Tokyo is 26 degrees celsius with rainy conditions.', error=False)

# Appendix A: Advanced Customization

You can safely skip this section, you won't need this.

#### Choosing Target Class - Method 2

You will only need it if you're adding tools not to the root class, otherwise, skip this


In [13]:
from baml_agents import ActionRunner

r = ActionRunner(TypeBuilder, b=b)
r.add_action(GetWeatherInfo)

question = "What's the weather like in the capital of Japan?"
request = r.b_(return_class="BamlCustomTools_NextAction").BamlCustomTools_GetNextAction(
    question
)
# OR
rc = types.BamlCustomTools_NextAction
request = r.b_(return_class=rc).request.BamlCustomTools_GetNextAction(question)

display_prompt(request)

[system]
Please select the next best action to achieve the goal.

‹goal›
What's the weather like in the capital of Japan?
‹/goal›

Answer in JSON format with exactly one next best action:
{
  chosen_action: {
    // Get weather information for a given city.
    action_id: "get_weather_info",
    city: string,
    measurement: "celsius" OR "fahrenheit" OR null,
  },
}


#### Choosing Target Class - Method 3

You will only need this so you could fall back to a simpler method if you encouter bugs or need more customization

In [14]:
from baml_agents import ActionRunner

r = ActionRunner(TypeBuilder)
r.add_action(GetWeatherInfo)

# Choose one of the following ways to create the type builder:
tb = r.tb("BamlCustomTools_NextAction")  # Either way works
tb = r.tb("BamlCustomTools_NextAction", tb=TypeBuilder())  # Either way works
tb = r.tb(types.BamlCustomTools_NextAction)  # Either way works
tb = r.tb(types.BamlCustomTools_NextAction, tb=TypeBuilder())  # Either way works

question = "What's the weather like in the capital of Japan?"
request = b.request.BamlCustomTools_GetNextAction(question, baml_options={"tb": tb})
display_prompt(request)

[system]
Please select the next best action to achieve the goal.

‹goal›
What's the weather like in the capital of Japan?
‹/goal›

Answer in JSON format with exactly one next best action:
{
  chosen_action: {
    // Get weather information for a given city.
    action_id: "get_weather_info",
    city: string,
    measurement: "celsius" OR "fahrenheit" OR null,
  },
}
